In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

import sys
sys.path.append("../..")  # 상위 디렉토리를 sys.path에 추가

import requests
import pandas as pd
from mySetting import Client_ID, Client_Secret

# 네이버 API 키
client_id = Client_ID  # 클라이언트 ID
client_secret = Client_Secret  # 클라이언트 Secret

def get_coordinates(address):
    url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'
    headers = {
        'X-NCP-APIGW-API-KEY-ID': client_id,
        'X-NCP-APIGW-API-KEY': client_secret,
    }
    params = {
        'query': address,
    }
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    if data['addresses']:
        x = data['addresses'][0]['x']
        y = data['addresses'][0]['y']
        return (x, y)
    else:
        return (None, None)

In [4]:
def get_amenity(new_park, new_library, new_rest):
    # 공원
    park = pd.read_csv('../../' + new_park, encoding='cp949')
    park = park[['공원명','소재지지번주소','위도','경도']].rename(columns={'소재지지번주소':'주소', '공원명':'이름'})


    # 도서관
    library = pd.read_csv('../../' + new_library, encoding='cp949')

    lib = library.loc[library['소재지주소'].str.contains('평택시')][['도서관명','소재지주소']].reset_index().drop(columns=['index'])
    lib['위도'] = pd.NA
    lib['경도'] = pd.NA

    for i, address in enumerate(lib['소재지주소']):
        lng, lat = get_coordinates(address)
        lib.at[i, '위도'] = lat
        lib.at[i, '경도'] = lng

    lib = lib.rename(columns={'소재지주소':'주소', '도서관명':'이름'})


    # 카페, 편의점
    all_rest = pd.read_excel('../../' + new_rest)

    tmp = all_rest[['사업장명','소재지전체주소','업태구분명']].reset_index().drop(columns=['index'])
    tmp['위도'] = pd.NA
    tmp['경도'] = pd.NA

    for i, address in enumerate(tmp['소재지전체주소']):
        lng, lat = get_coordinates(address)
        tmp.at[i, '위도'] = lat
        tmp.at[i, '경도'] = lng

    cafe = tmp.loc[tmp['업태구분명'] == '커피숍'].drop(columns=['업태구분명']).rename(columns={'소재지전체주소':'주소', '사업장명':'이름'})
    conv = tmp.loc[tmp['업태구분명'] == '편의점'].drop(columns=['업태구분명']).rename(columns={'소재지전체주소':'주소', '사업장명':'이름'})


    return park, lib, cafe, conv

In [6]:
new_park = 'new_data/편의시설/경기도_평택시_도시공원정보_20240308.csv'
new_library = 'new_data/편의시설/경기도공공도서관현황.csv'
new_rest = 'new_data/편의시설/휴게음식점.xlsx'

park, lib, cafe, conv = get_amenity(new_park, new_library, new_rest)

/Users/hdk/anaconda3/envs/qa_system/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [11]:
park.to_csv('../../preprocessed/편의시설/공원.csv', index=False)
lib.to_csv('../../preprocessed/편의시설/도서관.csv', index=False)
cafe.to_csv('../../preprocessed/편의시설/카페.csv', index=False)
conv.to_csv('../../preprocessed/편의시설/편의점.csv', index=False)